# Master Admission

Take a look at the data in "admit-train.csv". The attribute "admit" indicates whether a student has been admitted to a Master's course. The attributes "gre" and "gpa" contain the results of certain exams. The attribute "rank" represents the reputation rank of the student's current university, where smaller ranks correspond to higher reputation.

In [ ]:
import pandas as pd

# Read data into dataframe
df_train = pd.read_csv("admit-train.csv")

# Show data
df_train.head()

In [ ]:
# create pairplot and get an overview of variable dependencies and distributions
pd.plotting.scatter_matrix(df_train);

## a)

Name the dependent variable and the independent variables.  
Which scales of measurement do the variables belong to? (e.g. nominal, ordinal, interval or ratio)

**Solution**

| Variable | Dependent?  | Scale of measurement |
| -------- | ----------- | --------------------- |
| *"rank"* | Independent | Ordinal               |
| *"gre"*  | Independent | Interval              |
| *"gpa"*  | Independent | Interval              |
| *"admit"*| Dependent   | Nominal               |

## b)

Use the statsmodels module and train a generalized linear model on the data from "admit-train.csv". 

In [ ]:
import statsmodels.formula.api as smf

# Mark "rank" as categorical
df_train["rank"] = df_train["rank"].astype("category")

# Define and fit a model
logreg = smf.logit("admit ~ gre + gpa + rank", data=df_train)
result = logreg.fit()
print(result.summary())

**Observation**: The categorical variable "rank" is represented by three different variables which indicate whether the rank has a certain value:
$$
rank[T.i] = \mathbb{I}[rank = i]
$$
For $rank = 1$, we don't need another variable since this case can be inferred if all other rank-variables are zero.

Which coefficients are statistically significant for a level of $\alpha=0.05$? 

**Solution**

For most attributes, the test-statistic $z$ has a p-value (``P > |z|``) which is below $\alpha$. We can conclude that these attributes are statistically significant at the given signifance level. Only for the attribute ``rank[T.2]``, we cannot reject the null hypothesis $H_0: \beta_{rank = 2} = 0$.

## c)

Interpret the coefficients.

**Solution**

| Parameter | Value | Interpretation |
| --- | --- | --- |
| intercept  | -4.8711 | - |
| $\beta_{gre}$ | 0.0031 | As the "gre" score increases by one point, the odds of being admitted increase by a factor of 1.003. ($\exp(0.003) \approx 1.003$) |
| $10 \cdot \beta_{gre}$ | 0.0301 | As the "gre" score increases by ten points, the odds of being admitted increase by a factor of 1.03. ($\exp(0.03) \approx 1.03$) |
| $\beta_{gpa}$ | 0.9204 | As the "gpa" score increases by one point, the odds of being admitted increase by a factor of 2.51. ($\exp(0.92) \approx 2.51$) |
| $\beta_{rank = 2}$ | -0.6255 | If the rank is 2, the odds of being accepted is lowered by a factor of 0.53 ($\exp(-0.63) \approx 0.53$) compared to rank 1. |
| $\beta_{rank = 3}$ | -1.5245 | If the rank is 3, the odds of being accepted is lowered by a factor of 0.22 ($\exp(-1.52) \approx 0.22$) compared to rank 1. |
| $\beta_{rank = 4}$ | -1.6853 | If the rank is 4, the odds of being accepted is lowered by a factor of 0.18 ($\exp(-1.69) \approx 0.18$) compared to rank 1. |

## d)

Test the significance of the attribute "rank" by using a Wald test. 

Is the attribute "rank" statistically significant w.r.t. a level of $\alpha = 5\%$?

In [ ]:
# Specify the restrictions as a list of strings and pass it to "r_matrix"
wald_test_result = result.wald_test(
  	"(rank[T.2] = 0, rank[T.3] = 0, rank[T.4] = 0)", 
  	scalar=True
)
print(f"Test statistic (chi^2_{int(wald_test_result.df_denom)}-distributed): {wald_test_result.statistic}")
print(f"p-value of the statistic: {wald_test_result.pvalue}")

**Solution**

The attribute is statistically significant since $\text{p-value} < 5\%$.  
This means that we can reject the null-hypothesis ("$\beta_{\text{rank}}$ = 0") at a significance level of $\alpha = 5 \%$.

## e)

In order to gain a better understanding of the model, have a look at the predicted probabilities of some data points. Adjust only one parameter and keep the others constant. For example, keep "gre" and "gpa" constant (mean value), while varying "rank".

**Solution**

In [ ]:
rank = [1, 2, 3, 4]
new_points = pd.DataFrame({"gre" : df_train["gre"].mean(),
                          "gpa" : df_train["gpa"].mean(),
                          "rank" : rank})

new_points["predictions"] = result.predict(new_points)

print(new_points)


## f)

Find the McFadden ratio and interpret the results.

**Solution**

In [ ]:
# McFadden Ratio
print(result.prsquared)

## g)

Load the test data from *admit-test.csv* and predict the probability.

**Solution**

In [ ]:
# load test data
df_test = pd.read_csv("admit-test.csv")

# assign proper data type
df_test["rank"] = df_test["rank"].astype("category")

# make predictions on test data
y_pred_test = (result.predict(df_test.drop(columns = ["admit"])) > 0.5).astype(int)

Construct the confusion matrix for the test data.

**Solution**

In [ ]:
# build confusion matrix
y_true_test = df_test["admit"]
confusion_matrix = pd.crosstab(y_true_test, y_pred_test)
confusion_matrix

## h)

Compute the accuracy 
$$
\frac{TP + TN}{TP + TN + FP + FN}
$$
of your trained model on the test set.

**Solution**

In [ ]:
correct_predictions = (y_true_test == y_pred_test).sum()
total_predictions = len(y_true_test)
accuracy = correct_predictions / total_predictions

print("Accuracy =", "{:.3f}".format(accuracy))